<div style="text-align: center; border: 2px solid gray; padding: 10px; margin: 10px auto; width: fit-content;">
<font color='gray'>AI in Petroleum Industry</font><br>
<font color='gray'>Deep Neural Networks for Reservoir Production Forecasting</font><br><br>
By <font color='green'>Novin Nekuee</font> (403134029)  & <font color='green'>Soroosh Danesh</font> (123456789)<br>
<p>Dr. Emami & Eng. Nasiri</p>
</div>

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [7]:
df_raw = pd.read_excel('./assets/data.xlsx')

In [8]:
num_available_samples = 756
df_filtered = df_raw[df_raw['sample number'] <= num_available_samples].copy()

In [9]:
df_pivot = df_filtered.pivot_table(
    index='sample number',
    columns='Month (2026)',
    values=['Initial Sw', 'Oil Rate (m3/day)', 'Cumulative Oil (M m3)']
)

In [10]:
df_pivot.columns = [f'{val}_{month}' for val, month in df_pivot.columns]
df_pivot.reset_index(inplace=True)

In [11]:
sw_cols = [col for col in df_pivot.columns if 'Initial Sw' in col]
df_pivot['Initial Sw'] = df_pivot[sw_cols[0]]
df_pivot.drop(columns=sw_cols, inplace=True)

In [12]:
for col in df_pivot.columns:
    if df_pivot[col].isnull().any():
        mean_val = df_pivot[col].mean()
        df_pivot[col].fillna(mean_val, inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_30708\3285668716.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_pivot[col].fillna(mean_val, inplace=True)


In [13]:
display(df_pivot.head())

,sample number,Cumulative Oil (M m3)_1,Cumulative Oil (M m3)_3,Cumulative Oil (M m3)_5,Cumulative Oil (M m3)_7,Cumulative Oil (M m3)_9,Cumulative Oil (M m3)_11,Oil Rate (m3/day)_1,Oil Rate (m3/day)_3,Oil Rate (m3/day)_5,Oil Rate (m3/day)_7,Oil Rate (m3/day)_9,Oil Rate (m3/day)_11,Initial Sw
0,1,0.000681,25.4710,49.735,73.408,96.928,119.580,980.04,410.07,397.78,388.08,379.36,371.30,0.25
1,2,0.003307,118.8300,218.460,302.950,378.490,444.910,4762.30,1828.00,1578.20,1385.00,1218.40,1088.80,0.23
2,3,0.001104,45.5290,87.461,127.690,167.110,204.590,1590.00,725.02,687.42,659.53,635.76,614.49,0.21
3,4,0.003663,138.6700,259.880,369.820,470.760,558.220,5274.30,2199.30,1987.10,1802.20,1628.10,1433.70,0.25
4,5,0.000214,9.7412,19.257,28.656,38.117,47.345,307.56,159.06,156.00,154.09,152.59,151.28,0.22


In [14]:
df_pivot.to_csv('processed_tabular_data.csv', index=False)

In [17]:
perm_folder = './assets/permeability/'
poro_folder = './assets/porosity/'

num_samples = 756 
all_images_list = []

In [18]:
for i in range(1, num_samples + 1):
    # ساختن نام فایل با صفرهای ابتدایی (مثلاً '0001', '0756')
    sample_id = str(i).zfill(4)
    
    perm_path = os.path.join(perm_folder, f'perm_map_{sample_id}.tiff')
    poro_path = os.path.join(poro_folder, f'poro_map_{sample_id}.tiff')
    
    # خواندن تصاویر
    perm_img = Image.open(perm_path)
    poro_img = Image.open(poro_path)
    
    # تبدیل به آرایه NumPy
    perm_array = np.array(perm_img, dtype=np.float32)
    poro_array = np.array(poro_img, dtype=np.float32)
    
    # ترکیب دو نقشه در یک آرایه دو کاناله
    combined_image = np.stack([perm_array, poro_array], axis=-1)
    all_images_list.append(combined_image)

# تبدیل لیست به یک آرایه بزرگ NumPy
image_data = np.array(all_images_list)

# --- نرمال‌سازی داده‌های تصویری ---
# کانال اول (تراوایی)
min_ch0 = image_data[:, :, :, 0].min()
max_ch0 = image_data[:, :, :, 0].max()
image_data[:, :, :, 0] = (image_data[:, :, :, 0] - min_ch0) / (max_ch0 - min_ch0)

# کانال دوم (تخلخل)
min_ch1 = image_data[:, :, :, 1].min()
max_ch1 = image_data[:, :, :, 1].max()
image_data[:, :, :, 1] = (image_data[:, :, :, 1] - min_ch1) / (max_ch1 - min_ch1)


print("پردازش تصاویر با موفقیت انجام شد.")
print(f"شکل نهایی آرایه تصاویر: {image_data.shape}")
print(f"کمترین مقدار در آرایه: {image_data.min()}")
print(f"بیشترین مقدار در آرایه: {image_data.max()}")

پردازش تصاویر با موفقیت انجام شد.
شکل نهایی آرایه تصاویر: (756, 64, 64, 2)
کمترین مقدار در آرایه: nan
بیشترین مقدار در آرایه: nan


In [19]:
problematic_files = []

for i in range(1, 757):
    sample_id = str(i).zfill(4)
    perm_path = os.path.join(perm_folder, f'perm_map_{sample_id}.tiff')
    poro_path = os.path.join(poro_folder, f'poro_map_{sample_id}.tiff')
    
    # بررسی فایل تراوایی
    try:
        img_array = np.array(Image.open(perm_path))
        if img_array.min() == img_array.max():
            problematic_files.append(perm_path)
    except FileNotFoundError:
        pass # اگر فایلی وجود نداشت، رد شو

    # بررسی فایل تخلخل
    try:
        img_array = np.array(Image.open(poro_path))
        if img_array.min() == img_array.max():
            problematic_files.append(poro_path)
    except FileNotFoundError:
        pass

if problematic_files:
    print("\nفایل‌های زیر ممکن است خالی یا یکنواخت باشند:")
    for f in problematic_files:
        print(f)
else:
    print("هیچ فایل یکنواختی پیدا نشد.")

هیچ فایل یکنواختی پیدا نشد.


In [20]:

num_samples = 756
all_images_list = []

print(f"شروع به خواندن {num_samples} جفت تصویر...")

# --- خواندن و ترکیب تصاویر در یک حلقه ---
for i in range(1, num_samples + 1):
    sample_id = str(i).zfill(4)
    perm_path = os.path.join(perm_folder, f'perm_map_{sample_id}.tiff')
    poro_path = os.path.join(poro_folder, f'poro_map_{sample_id}.tiff')
    
    perm_img = Image.open(perm_path)
    poro_img = Image.open(poro_path)
    
    perm_array = np.array(perm_img, dtype=np.float32)
    poro_array = np.array(poro_img, dtype=np.float32)
    
    combined_image = np.stack([perm_array, poro_array], axis=-1)
    all_images_list.append(combined_image)

image_data = np.array(all_images_list)

# --- نرمال‌سازی مقاوم در برابر خطای تقسیم بر صفر ---
# کانال اول (تراوایی)
min_ch0 = image_data[:, :, :, 0].min()
max_ch0 = image_data[:, :, :, 0].max()
# شرط برای جلوگیری از تقسیم بر صفر
if (max_ch0 - min_ch0) != 0:
    image_data[:, :, :, 0] = (image_data[:, :, :, 0] - min_ch0) / (max_ch0 - min_ch0)

# کانال دوم (تخلخل)
min_ch1 = image_data[:, :, :, 1].min()
max_ch1 = image_data[:, :, :, 1].max()
# شرط برای جلوگیری از تقسیم بر صفر
if (max_ch1 - min_ch1) != 0:
    image_data[:, :, :, 1] = (image_data[:, :, :, 1] - min_ch1) / (max_ch1 - min_ch1)

# مقادیر nan باقی‌مانده را با صفر جایگزین می‌کنیم (برای اطمینان)
image_data = np.nan_to_num(image_data)

print("\nپردازش تصاویر با موفقیت (نسخه اصلاح‌شده) انجام شد.")
print(f"شکل نهایی آرایه تصاویر: {image_data.shape}")
print(f"کمترین مقدار در آرایه: {image_data.min()}")
print(f"بیشترین مقدار در آرایه: {image_data.max()}")

شروع به خواندن 756 جفت تصویر...

پردازش تصاویر با موفقیت (نسخه اصلاح‌شده) انجام شد.
شکل نهایی آرایه تصاویر: (756, 64, 64, 2)
کمترین مقدار در آرایه: 0.0
بیشترین مقدار در آرایه: 1.0


In [ ]:
# --- هماهنگ‌سازی تعداد نمونه‌ها ---
# ابتدا لیست شماره نمونه‌های معتبری که در فایل اکسل وجود دارند را استخراج می‌کنیم
valid_sample_numbers = df_pivot['sample number'].values

# از آنجایی که شماره نمونه‌ها از 1 شروع می‌شود ولی اندیس آرایه از 0، یکی از آنها کم می‌کنیم
valid_indices = valid_sample_numbers - 1

# حالا آرایه تصاویر را بر اساس این اندیس‌های معتبر فیلتر می‌کنیم
X_image_filtered = image_data[valid_indices]


# --- جدا کردن ورودی‌ها (X) از خروجی‌ها (y) ---
# ورودی تصویری ما، آرایه فیلترشده است
X_image = X_image_filtered

# ورودی عددی، ستون 'Initial Sw'
X_numerical = df_pivot['Initial Sw'].values.reshape(-1, 1)

# خروجی‌ها (y) ستون‌های تولید هستند
target_cols = [col for col in df_pivot.columns if col not in ['sample number', 'Initial Sw']]
y = df_pivot[target_cols].values

# --- بررسی مجدد ابعاد قبل از تقسیم ---
print("ابعاد داده‌ها پس از هماهنگ‌سازی و قبل از تقسیم:")
print(f"  - تصاویر:   {X_image.shape}")
print(f"  - عددی:      {X_numerical.shape}")
print(f"  - خروجی‌ها:    {y.shape}")
print("-" * 40)


X_train_img, X_test_img, X_train_num, X_test_num, y_train, y_test = train_test_split(
    X_image, X_numerical, y, 
    test_size=0.2,
    random_state=42
)

X_train_image, X_val_image, X_train_number, X_val_number, y_train, y_val = train_test_split(


)

print("\nابعاد داده‌های آموزشی (Training):")
print(f"  - تصاویر (X_train_img): {X_train_img.shape}")
print(f"  - عددی (X_train_num):   {X_train_num.shape}")
print(f"  - خروجی‌ها (y_train):     {y_train.shape}")
print("\nابعاد داده‌های آزمایشی (Test):")
print(f"  - تصاویر (X_test_img):  {X_test_img.shape}")
print(f"  - عددی (X_test_num):    {X_test_num.shape}")
print(f"  - خروجی‌ها (y_test):      {y_test.shape}")

ابعاد داده‌ها پس از هماهنگ‌سازی و قبل از تقسیم:
  - تصاویر:   (753, 64, 64, 2)
  - عددی:      (753, 1)
  - خروجی‌ها:    (753, 12)
----------------------------------------

ابعاد داده‌های آموزشی (Training):
  - تصاویر (X_train_img): (602, 64, 64, 2)
  - عددی (X_train_num):   (602, 1)
  - خروجی‌ها (y_train):     (602, 12)

ابعاد داده‌های آزمایشی (Test):
  - تصاویر (X_test_img):  (151, 64, 64, 2)
  - عددی (X_test_num):    (151, 1)
  - خروجی‌ها (y_test):      (151, 12)


In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, concatenate

# --- 1. شاخه پردازش تصاویر (CNN) ---
# تعریف ورودی تصویر با ابعاد مناسب
image_input = Input(shape=(64, 64, 2), name='image_input')

# لایه‌های کانولوشنی برای استخراج ویژگی‌های فضایی
cnn = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(image_input)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
cnn = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)

# تبدیل خروجی CNN به یک بردار یک بعدی
cnn_flatten = Flatten()(cnn)


# --- 2. شاخه پردازش داده عددی (Dense) ---
# تعریف ورودی عددی (که فقط یک ویژگی دارد)
numerical_input = Input(shape=(1,), name='numerical_input')

# یک لایه کاملاً متصل برای پردازش این ویژگی
dense_num = Dense(units=8, activation='relu')(numerical_input)


# --- 3. ادغام دو شاخه ---
# ترکیب بردارهای ویژگی از دو شاخه
combined_features = concatenate([cnn_flatten, dense_num])


# --- 4. لایه‌های خروجی ---
# یک لایه Dense برای یادگیری الگوهای ترکیبی
final_dense = Dense(units=64, activation='relu')(combined_features)
# لایه خروجی نهایی با 12 نورون (برای 12 متغیر هدف)
# activation='linear' برای مسائل رگرسیون مناسب است
output = Dense(units=12, activation='linear', name='output')(final_dense)


# --- ساخت و کامپایل مدل نهایی ---
# تعریف مدل با مشخص کردن ورودی‌ها و خروجی
model = Model(inputs=[image_input, numerical_input], outputs=output)

# کامپایل مدل با مشخص کردن بهینه‌ساز و تابع هزینه
# 'adam' یک بهینه‌ساز محبوب و 'mean_squared_error' یک تابع هزینه استاندارد برای رگرسیون است
model.compile(optimizer='adam', loss='mean_squared_error')

# --- نمایش خلاصه معماری مدل ---
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 64, 64, 2) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 62, 62,    │        608 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 31, 31,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 29, 29,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 14, 14,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical_input     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 12544)     │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 8)         │         16 │ numerical_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 12552)     │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │    803,392 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 12)        │        780 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 823,292 (3.14 MB)

 Trainable params: 823,292 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# --- شروع فرآیند آموزش ---

# تعداد دوره‌های آموزش (برای شروع عدد کمی را انتخاب می‌کنیم)
epochs = 150
# تعداد نمونه‌هایی که در هر مرحله به مدل داده می‌شود
batch_size = 32

print("شروع آموزش مدل...")

# تابع fit فرآیند آموزش را آغاز می‌کند
history = model.fit(
    # ورودی‌های مدل را به صورت یک دیکشنری به آن می‌دهیم
    # چون ورودی‌های ما نام‌گذاری شده‌اند (image_input, numerical_input)
    x={'image_input': X_train_img, 'numerical_input': X_train_num},
    # خروجی‌های هدف
    y=y_train,
    # داده‌های ارزیابی (مجموعه آزمایشی)
    validation_data=(
        {'image_input': X_test_img, 'numerical_input': X_test_num}, 
        y_test
    ),
    # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    epochs=epochs,
    batch_size=batch_size,
    # نمایش نوار پیشرفت در حین آموزش
    verbose=1
)

print("\nآموزش مدل با موفقیت به پایان رسید!")

شروع آموزش مدل...
Epoch 1/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 5573912.0000 - val_loss: 9556526.0000
Epoch 2/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 4970021.0000 - val_loss: 8755401.0000
Epoch 3/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 3371653.5000 - val_loss: 5965384.5000
Epoch 4/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 3568600.0000 - val_loss: 4318698.0000
Epoch 5/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 2168177.2500 - val_loss: 2996121.2500
Epoch 6/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 1113801.8750 - val_loss: 1519426.1250
Epoch 7/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 1185829.6250 - val_loss: 865693.0625
Epoch 8/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 608215.8750 - val_loss: 854927.3125
Epoch 9/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 598769.8125 - val_loss: 829365.5000
Epoch 10/150
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 619930.8750 - val_loss: 808473.5625
Epoch 11